In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-motor-claims-datasets-fremtpl2freq/freMTPL2freq.csv


# Load the data

In [28]:
df = pd.read_csv('/kaggle/input/french-motor-claims-datasets-fremtpl2freq/freMTPL2freq.csv')

In [29]:
df.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


# Exploratory Data Analysis

The definitions of each column:
- IDpol: Policy ID, used to link to severity data set
- ClaimNb: Claim numbers during exposure period
- Exposure: Period of exposure for a policy in year(s)
- VehPower: Car power, in categorical format
- VehAge: Car age, in continuous value format
- DrivAge: Driver’s age in years, in continuous value format
- BonusMalus: Bonus / Malus rating, <100 means bonus, >100 means malus, in continous value format
- VehBrand: Car brand, in categorical format
- VechGas: Fuel type, in categorical format
- Area: Density rating of the area car driver lives in, in categorical format
- Density: Density rating, number of inhabitants per km^2, in continous value format
- Region: Policy region in France, in categorical format

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IDpol       678013 non-null  float64
 1   ClaimNb     678013 non-null  int64  
 2   Exposure    678013 non-null  float64
 3   Area        678013 non-null  object 
 4   VehPower    678013 non-null  int64  
 5   VehAge      678013 non-null  int64  
 6   DrivAge     678013 non-null  int64  
 7   BonusMalus  678013 non-null  int64  
 8   VehBrand    678013 non-null  object 
 9   VehGas      678013 non-null  object 
 10  Density     678013 non-null  int64  
 11  Region      678013 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 62.1+ MB


In [32]:
df.shape

(678013, 12)

Change IDpol from `float` to `object`. This is so that IDpol won't get picked up if we use a `describe()` method as it will be meaningless.

In [33]:
df['IDpol'] = df['IDpol'].astype('object')

In [7]:
df['ClaimNb'].value_counts()

ClaimNb
0     643953
1      32178
2       1784
3         82
4          7
11         3
5          2
6          1
8          1
16         1
9          1
Name: count, dtype: int64

### Missing any values

In [6]:
df.isnull().sum()

IDpol         0
ClaimNb       0
Exposure      0
Area          0
VehPower      0
VehAge        0
DrivAge       0
BonusMalus    0
VehBrand      0
VehGas        0
Density       0
Region        0
dtype: int64

### There is no missing values in the dataset

In [34]:
df.describe()

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density
count,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000
mean,0.053247,0.528750,6.454631,7.044265,45.499122,59.761502,1792.422405
std,0.240117,0.364442,2.050906,5.666232,14.137444,15.636658,3958.646564
min,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,1.000000
25%,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,92.000000
50%,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,393.000000
75%,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,1658.000000
max,16.000000,2.010000,15.000000,100.000000,100.000000,230.000000,27000.000000


### We will also simplify the data for our GLM model. In particular, we will adjust the following columns:

- ClaimNb: cap at 4 claims
- VehAge: cap at 20 years
- DrivAge: cap at 90 years old
- BonusMalus: cap at 150, round to nearest integer
- Density: apply log
- Exposure: cap at 1 year

and subsequently add the following 'GLM' columns that will be used for model fitting:
- AreaGLM: convert alphabet into integer
- VehPowerGLM: cap at 9
- VehAgeGLM: create 3 bins
- DrivAgeGLM: create 7 bins


One of the reasons for capping variables is that insurance claim counts are highly skewed: most policyholders have 0-2 claims, but a few may have 10+. Extreme values like 20 claims are rare, often due to data errors, fraud, etc. 

In [35]:
import math
from copy import deepcopy

In [37]:
df_freq = deepcopy(df)
df_freq['ClaimNb'] = df_freq['ClaimNb'].clip(upper=4)
df_freq['VehAge'] = df_freq['VehAge'].clip(upper=20)
df_freq['DrivAge'] = df_freq['DrivAge'].clip(upper=90)
df_freq['BonusMalus'] = df_freq['BonusMalus'].apply(lambda x: 150 if x > 150 else int(x))
df_freq['Density'] = df_freq['Density'].apply(lambda x: round(math.log(x), 2))
df_freq['Exposure'] = df_freq['Exposure'].clip(upper=1.0)
df_freq['AreaGLM']=df_freq['Area'].apply(lambda x: ord(x)-64)
df_freq['VehPowerGLM']=df_freq['VehPower'].apply(lambda x: 9 if x > 9 else x)
df_freq['VehPowerGLM']=df_freq['VehPowerGLM'].apply(lambda x: str(x))
df_freq['VehAgeGLM']=pd.cut(df_freq['VehAge'], bins=[0,1,10,np.inf], labels=[1,2,3], include_lowest=True)
df_freq['DrivAgeGLM']=pd.cut(df_freq['DrivAge'], bins=[18,21,26,31,41,51,71,np.inf], labels=[1,2,3,4,5,6,7], include_lowest=True)
df_freq['BonusMalusGLM'] = df_freq['BonusMalus']
df_freq['DensityGLM'] = df_freq['Density']

In [38]:
df_freq.describe()

,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density,AreaGLM,BonusMalusGLM,DensityGLM
count,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000,678013.000000
mean,0.053179,0.528545,6.454631,6.976124,45.496871,59.757211,5.981583,3.289698,59.757211,5.981583
std,0.237954,0.364081,2.050906,5.398963,14.129872,15.607906,1.870562,1.382685,15.607906,1.870562
min,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,0.000000,1.000000,50.000000,0.000000
25%,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,4.520000,2.000000,50.000000,4.520000
50%,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,5.970000,3.000000,50.000000,5.970000
75%,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,7.410000,4.000000,64.000000,7.410000
max,4.000000,1.000000,15.000000,20.000000,90.000000,150.000000,10.200000,6.000000,150.000000,10.200000


In [39]:
df_freq.columns

Index(['IDpol', 'ClaimNb', 'Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge',
       'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region', 'AreaGLM',
       'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM', 'BonusMalusGLM',
       'DensityGLM'],
      dtype='object')